<a href="https://colab.research.google.com/github/interactive-fiction-class/interactive-fiction-class.github.io/blob/master/homeworks/schemas/HW5_Schemas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 5: COMET-ATOMIC Schema

In this tutorial, you will create your own schema to represent the state of a story world as it goes through the story line by line.
A **schema** is a structured reprensentation made to hold facts or a plan, which in this case, can be used to track change over time.

**The purpose of this tutorial is to test your understanding of schemas and get hands-on experience with a state-of-the-art tool in commonsense reasoning.**


## Our Task

Let's teach our agent some basic information about the world!

------------------------

Formally, the task is:

Given an input sentence at time *t* (*In_t*), produce a schema *S_t*. Do this for each sentence in the story.

For example, using VerbNet:

| Timestep | Input | Schema |
|--------|-------|--------|
| 1 | Bethany picks up the sword. | `Bethany: has_possesion(sword)` |
| 2 | Bethany throws the sword. | `Bethany: !has_possesion(sword)` |

*In_1* - "Bethany picks up the sword."

would produce

*S_1* - `Bethany: has_possesion(sword)`

But then if the next sentence is:

*In_2* - "Bethany throws the sword."

The state would be updated to

*S_2* - `Bethany: !has_possesion(sword)`

-----------------------------
Your resulting system will be sort of like this simplified diagram (the parser is provided for you):
![Given a sentence_t and the knowledge representation from your knowledge database of choice, produce schemas (via some schema processor you create)](https://interactive-fiction-class.org/homeworks/schemas/schemas.png)

There is some knowledge about the story world, and you are using a schema to feed this information into bite-sized chunks so that your agent (and you) can understand it.
You will then have a processing step on the schema where you update it as you get more information as the story progresses.


To reiterate, you will:
1. Get to know [COMET-ATOMIC-2020](https://aaai.org/ojs/index.php/AAAI/article/view/4160). (Alternate link in case AAAI is down: https://arxiv.org/abs/2010.05953)
2. Make a __schema manipulator__ (not a real term, but I think it sounds cool), which will take in knowledge from ATOMIC and spit out your schema. Skeleton code is provided for you, but you are welcome to change things so that it makes more sense to you. This involves two steps:

  a. Validate: Generate preconditions to determine if an event can be added.

  b. Update: Generate effects to update your world state.  



# What is ATOMIC?

ATOMIC is a commonsense knowledge graph with some social inferences, among other things.

```
@inproceedings{sap2019atomic,
   title={ATOMIC: An Atlas of Machine Commonsense for If-Then Reasoning},
   author={Sap, Maarten and LeBras, Ronan and Allaway, Emily and Bhagavatula, Chandra and Lourie, Nicholas and Rashkin, Hannah and Roof, Brendan and Smith, Noah A and Choi, Yejin},
   year={2019},
   booktitle={AAAI},
   url={https://aaai.org/ojs/index.php/AAAI/article/view/4160}
}
```

It contains the following inferences about people and events:

* Because PersonX wanted (xIntent)
* Before, PersonX needed (xNeed, HasPrerequisite)
* PersonX is seen as (xAttr)
* As a result, PersonX feels (xReact)
* As a result, PersonX wants (xWant)
* As a result, PersonX reasons (xReason)
* PersonX then (xEffect)
* As a result, others feel (oReact)
* As a result, others want (oWant)
* Others then (oEffect)
* Happens before (isBefore)
* Happens after (isAfter)
* Is hindered by (HinderedBy)
* Causes (Causes)

and inferences about entities:
* Is located at (AtLocation, LocatedNear, LocationOfAction)
* Is made up of (MadeUpOf, PartOf. NotMadeOf)
* Is used to (UsedFor, ObjectUse)
* Has the property (HasProperty, NotHasProperty)
* Is capable of (CapableOf, NotCapableOf)
* Desires (Desires, NotDesires)


Among other things (full list is in the `all_relations` variable below).


[COMET](https://aclanthology.org/P19-1470) is the model that is trained on ATOMIC.

#Setup

## Get COMET and Install packages

Repo: https://github.com/allenai/comet-atomic-2020/

In [ ]:
# Clone the repo
%%capture
!git clone https://github.com/allenai/comet-atomic-2020.git

In [ ]:

# Enter the directory
import os
os.chdir('comet-atomic-2020')

In [ ]:
# Install ATOMIC's dependencies
%%capture
!pip install rouge_score
!pip install transformers
!pip install -r requirements.txt

### Also, Stanford's Stanza Parser

In order to process the input sentences, you will need to do some parsing. Here, we have setup Stanford's English language NER (Named Entity Recognition) and constituency parser using [Stanza](https://stanfordnlp.github.io/stanza/index.html). You're also welcome to use any of the other parsers they provide (or even switch to a completely different type of parser that you like better).

In [ ]:
%%capture
!python -m pip install stanza

import stanza
stanza.download('en')

**You might need to restart the runtime now.**


In [ ]:
# Example code to run Stanza

import stanza #here's the re-import for when your runtime is restarted
import json
nlp = stanza.Pipeline('en', processors='tokenize, ner, mwt') #lemma, depparse, constituency, pos
parse = nlp("Aurora submitted her resignation to Facebook.")
y = json.loads(str(parse))
y

## COMET-ATOMIC-2020 (BART) Setup

In [ ]:
# Download the model
%%capture
!bash models/comet_atomic2020_bart/download_model.sh

**Tip: Take note of the `all_relations` dictionary below! You will need it later on.**

In [ ]:
# Load the model
# copied from models/comet_atomic2020_bart/generation_example.py

import json
import torch
import argparse
from tqdm import tqdm
from pathlib import Path
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from models.comet_atomic2020_bart.utils import calculate_rouge, use_task_specific_params, calculate_bleu_score, trim_batch


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


class Comet:
    def __init__(self, model_path):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        task = "summarization"
        use_task_specific_params(self.model, task)
        self.batch_size = 1
        self.decoder_start_token_id = None

    def generate(
            self,
            queries,
            decode_method="beam",
            num_generate=5,
            ):

        with torch.no_grad():
            examples = queries

            decs = []
            for batch in list(chunks(examples, self.batch_size)):

                batch = self.tokenizer(batch, return_tensors="pt", truncation=True, padding="max_length").to(self.device)
                input_ids, attention_mask = trim_batch(**batch, pad_token_id=self.tokenizer.pad_token_id)

                summaries = self.model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_start_token_id=self.decoder_start_token_id,
                    num_beams=num_generate,
                    num_return_sequences=num_generate,
                    )

                dec = self.tokenizer.batch_decode(summaries, skip_special_tokens=True, clean_up_tokenization_spaces=False)
                decs.append(dec)

            return decs


all_relations = [
    "AtLocation",
    "CapableOf",
    "Causes",
    "CausesDesire",
    "CreatedBy",
    "DefinedAs",
    "DesireOf",
    "Desires",
    "HasA",
    "HasFirstSubevent",
    "HasLastSubevent",
    "HasPainCharacter",
    "HasPainIntensity",
    "HasPrerequisite",
    "HasProperty",
    "HasSubEvent",
    "HasSubevent",
    "HinderedBy",
    "InheritsFrom",
    "InstanceOf",
    "IsA",
    "LocatedNear",
    "LocationOfAction",
    "MadeOf",
    "MadeUpOf",
    "MotivatedByGoal",
    "NotCapableOf",
    "NotDesires",
    "NotHasA",
    "NotHasProperty",
    "NotIsA",
    "NotMadeOf",
    "ObjectUse",
    "PartOf",
    "ReceivesAction",
    "RelatedTo",
    "SymbolOf",
    "UsedFor",
    "isAfter",
    "isBefore",
    "isFilledBy",
    "oEffect",
    "oReact",
    "oWant",
    "xAttr",
    "xEffect",
    "xIntent",
    "xNeed",
    "xReact",
    "xReason",
    "xWant",
    ]

print("model loading ...")
comet = Comet("./comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")



Run your queries. `head` is the input sentence, and `rel` is the relation, as seen in `all_relations`.

In [ ]:
# Example code to run COMET. A method has been written for you further down called callCOMET().
queries = []
head = "PersonX relies on PersonY"
rel = "xNeed"
query = "{} {} [GEN]".format(head, rel)
queries.append(query)
print(queries)
results = comet.generate(queries, decode_method="beam", num_generate=5)
print(results)

In [ ]:
"""
# Optional: Get ATOMIC data (if you wanted the train/test/val sets)
!wget https://ai2-atomic.s3-us-west-2.amazonaws.com/data/atomic2020_data-feb2021.zip
!unzip atomic2020_data-feb2021.zip
"""

# Parse the sentence to feed into COMET
You're welcome to change this to fit your needs

In [ ]:
import nltk
from nltk.tree import Tree
from collections import Counter, defaultdict


class SentParser:
  """
  Parse the sentence and get the entities and the new phrase with tags
  """
  def __init__(self, sentence):
    sentence = sentence.replace(".","")
    parse = nlp(sentence) # call Stanza
    self.phrase = sentence # original sentence
    self.entities = dict() # dict of labels for entities e.g., [{PersonX: John}]
    self.new_phrase = sentence # the sentence with person names changed to PersonX and PersonY

    for sentence in parse.sentences:
      ents = self.getEntities(sentence.tokens)
      self.entities = ents

      for tag in ents.keys():
        person = ents[tag]
        self.new_phrase = self.new_phrase.replace(person, tag)

  def getEntities(self, parse):
    """
    get the named entities so you can pass it to ATOMIC's input and
    fill the PersonX and PersonY tags from the output

    args:
    parse (list) - list of Stanza token objects for this phrase

    return:
    entities (dict) - keeps track of who is PersonX and PersonY e.g. {PersonX: John}
    """
    entities = dict()
    count = 0
    for word in parse:
      if "PERSON" in word.ner:
        if count == 0:
          entities['PersonX'] = word.text
          count+=1
        elif count == 1:
          entities['PersonY'] = word.text
    return entities


In [ ]:
# Example call
s = SentParser("John went to the bank.")
print(s.phrase)
print(s.entities)
print(s.new_phrase)

# TODO: Setup your schema

Use a subset of the relations from `all_relations` for what should be a pre-condition and what should be an effect (or both!). Work with whatever you think makes sense.

Tip: You might want to not take every single fact that ATOMIC gives you. Try to come up with a heuristic to just take what you need.

In [ ]:
# for computing embeddings
!pip install -U -q sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# subset relations

nprecon_relat = ['NotHasProperty',
                 'HinderedBy',
                 'NotCapableOf',
                 'NotMadeOf',
                 'NotDesires',
                 ]

precon_relat = ['LocatedNear',
                'LocationOfAction',
                'UsedFor',
                'xNeed',
                'HasPrerequisite',
                'xAttr',
                'isAfter',
                'ObjectUse',
                'HasProperty',
                'CapableOf',
                'Desires',
                'AtLocation',
                'xIntent',
                'MadeUpOf',
                'PartOf',
                          ]

eff_relat = ['xReact',
             'oReact',
             'xWant',
             'oWant',
             'xEffect',
             'oEffect',
             'Causes',
             'isBefore',
             'xReason',
            ]

In [ ]:
# final code

def callCOMET(sent, rel, decode="beam", num=5):
  """
  Making COMET generate facts based on an input sent and a relation rel
  You can also provide the decoding method and the number of facts
  you want it to output.
  """
  query = ["{} {} [GEN]".format(sent, rel)]
  gen = comet.generate(query, decode_method=decode, num_generate=num)[0]
  return [s.strip().replace(".","") for s in gen if s.strip() != "none" or s.strip() != "."]

def fillEntityTags(fact, NER):
  """
  Given a output fact from COMET (str) and the NER (dict), replace the PersonX/Y
  tags with their original names
  """
  new = fact
  for entity in NER.keys():
    new = new.replace(entity,NER[entity])
  return new

class Predicate:
  """
  Individual precondition and effect objects
  """
  def __init__(self, rel, statement, isPre, neg=False):
    self.relation = rel # string - input COMET relation
    self.statement = statement # string - results from COMET
    self.isPrecondition = isPre # boolean - if it's a precondition (True) or an effect (False)
    self.isNegated = neg # boolean - if it's negated (True) or not (False)

class Schema:
  """
  Your schema
  """
  def __init__(self, starting_state):
    self.state = starting_state # defaultdict(set) - a set of facts for each entity
    # print("Initial state:", self.state)
    self.curr_event = "" # string - received from the parser; phrase from the original input sentence with PersonX/PersonY labels
    self.curr_NER = dict() # dict - person names and their corresponding tags for a given subevent/phrase
    self.preconditions = [] # list - stores Predicate objects
    self.timestep = 0 # int - how far you are in the story (optional)

  ### Check the preconditions against the state ###
  def checkPrecondition(self, pred):
    """
    Given this precondition and the current state, does this precondition pass?
    args:
    pred (Predicate)

    return:
    boolean - whether or not this event is valid
    """
    if pred.isPrecondition == False: return False #it's an effect, don't consider it

    ### Your code here ###
    #TODO: check pred against self.state
    # You can do direct matches or "close enough" (i.e., similarity) matches
    #e.g.:
    '''
    if pred.statement not in self.state[self.curr_NER['PersonX']]:
      return False
    '''
    ######################

    # Retrieve facts for 'PersonX' from the current state
    state_facts = list(self.state[self.curr_NER['PersonX']])

    # Prepare input lists for computing embeddings
    precon_list = [pred.statement] * len(state_facts)

    # Compute embeddings
    precon_embed = model.encode(precon_list, convert_to_tensor=True)
    state_embed = model.encode(state_facts, convert_to_tensor=True)

    # Compute cosine similarity
    cos_scores = util.cos_sim(precon_embed, state_embed)
    score = torch.any(cos_scores > 0.6)

    return score


  def getPreconditions(self):
    """
    Given the input event string (self.curr_event),
    return a list of preconditions (list of Predicate objects)
    """
    pre = []

    ### Your code here ###
    #TODO: complete the list using the **relevant** relations from "all_relations"
    #an example:
    '''
    rel = "HasPrerequisite"
    comet_out = callCOMET(self.curr_event,rel)
    for fact in comet_out:
      filled = fillEntityTags(fact,self.curr_NER)
      pre.append(Predicate(rel, filled, True)) #Predicate(rel, statement, isPrecondition, negated)
    ######################
    '''
    # define relevant relations

    precon_relat =['LocatedNear',
                'LocationOfAction',
                'UsedFor',
                'xNeed',
                'HasPrerequisite',
                'xAttr',
                'isAfter',
                'ObjectUse',
                'HasProperty',
                'CapableOf',
                'Desires',
                'AtLocation',
                'xIntent',
                'MadeUpOf',
                'PartOf',
                  ]

    nprecon_relat =['NotHasProperty',
                 'HinderedBy',
                 'NotCapableOf',
                 'NotMadeOf',
                 'NotDesires',
                   ]

    for rel in precon_relat:
      comet_out = callCOMET(self.curr_event,rel)
      for fact in comet_out:
        filled = fillEntityTags(fact,self.curr_NER)
        pre.append(Predicate(rel, filled, True)) #Predicate(rel, statement, isPrecondition, negated)

    # for opposite of relation
    for rel in nprecon_relat:
      comet_out = callCOMET(self.curr_event,rel)
      for fact in comet_out:
        filled = fillEntityTags(fact,self.curr_NER)
        pre.append(Predicate(rel, filled, False)) #Predicate(rel, statement, isPrecondition, negated)

    return pre


  def checkValidity(self):
    """
    Goes through all the preconditions to check to see if
    this event can be added to the state.
    A precondition is considered valid as long as

    return:
    boolean - whether or not this event is valid
    """
    print("State:",self.state)
    preconds = self.getPreconditions()
    valids = []
    for precond in preconds:
      print(precond.statement)
      valid = self.checkPrecondition(precond)
      print("Valid",valid)
      if valid:
        valids.append(precond)
    if valids:
      self.preconditions += valids
      return True
    return False


  ### Once validated, update the schema ###
  def getEffects(self):
    """
    Given the input event string (self.curr_event),
    return a list of effects (list of Predicate objects)
    """
    effects = []

    ### Your code here ###
    #TODO: complete the list using the **relevant** relations from "all_relations"
    #an example:
    # rel = "xReact"
    # define relevant relations

    eff_relat =['xReact',
             'oReact',
             'xWant',
             'oWant',
             'xEffect',
             'oEffect',
             'Causes',
             'isBefore',
             'xReason',
               ]

    for rel in eff_relat:
        fact = callCOMET(self.curr_event,rel)[0]
        filled = fillEntityTags(fact, self.curr_NER)
        effects.append(Predicate(rel, filled, False)) #Predicate(rel, statement, isPrecondition, negated)
    ######################

    return effects


  def updateSchema(self, event, NER_dict):
    """
    Given an input event string (event), check the validity of adding it to the state,
    and update the schema state (self.state) with new effects
    args:
    event(str) - received from the parser; phrase from the original input sentence with PersonX/PersonY labels
    NER_dict(dict) - person names and their corresponding tags for a given subevent/phrase, e.g. NER_dict['PersonX'] = "Cindy"
    """
    self.curr_event = event
    self.curr_NER = NER_dict

    #1) check validity
    valid = self.checkValidity()
    print("Preconditions:",self.preconditions[0].statement)

    #2) update the state
    effects = self.getEffects()
    print("Effects:", effects[0].statement)


    ### Your code here ###

    #TODO: add these new effects to the state
    for effect in effects:
      # changed code for every effect in the defined eff_relat list
      for effect.relation in eff_relat:
        self.state[NER_dict['PersonX']].add(effect.statement)

    # TODO: (extra credit) remove facts that aren't true anymore
    # Although this is ideal, it might be hard to figure out when facts are negated

    ######################





## Story Tracking Questions
Run the following stories through your system and print out your schema after each sentence (or subevent if there are multiple events in a sentence).
For each scenario, **keep the print out of your schema for that story in your ipynb**.

**Do not change your schema code in between running these examples! Your final schema code should be able to run multiple scenarios.**

In [ ]:
# Function for updating the schema throughout a story
def runStory(story, start = defaultdict(set)):
  schema = Schema(start)
  for sent in story:
    print(sent)
    s = SentParser(sent)
    try:
      schema.updateSchema(s.new_phrase,s.entities)
      print("Schema:",schema.state)
    except:
      print("Story fails!")

In [ ]:
# Testing call 1
start = defaultdict(set)
start.update({"John": set(["John is hungry."])})
runStory(["John eats an apple."], start)

In [ ]:
# Testing call 2
start = defaultdict(set)
start.update({"Imani": set(["Imani has a toothache."])})
runStory(["Imani made an appointment to see the dentist.", "The dentist told Imani that she had a cavity.", "Imani never had a cavity before.", "Imani was not looking forward to her next dentist appointment."], start)

Please keep the following code blocks commented out until you finish your schema.

In [ ]:
stories = {
    1: ["Gina misplaced her phone.", "Gina looks for her phone in the living room.", "Gina remembers leaving her phone in the car.", "Gina goes back to the car.", "Gina finds her phone in the car."],
    2: ["Phil was at the community pool.","Phil thought he could go out to the deeper end by himself.","Phil jumps into the deep end.","Phil has trouble staying afloat.","The lifeguard had to help Phil out of the water."],
    3: ["Amy was happy her first class in junior high was all new kids.", "Amy introduced herself to the girl seated next to her.", "The girl was even more nervous than Amy to make friends.", "The girls talked and bonded over their love of books.", "The girls decided to meet up after school to go to the library."],
    4: ["Xander's dog hates his treats.", "Xander decided to go buy some new dog treats.", "None of the dog treats at the pet store looked tasty.", "Xander decided to buy his dog some salmon from the fish market.", "Xander's dog loved the salmon."],
    5: ["Franco has never cooked for his family.", "Franco decided to follow an old family recipe.", "Franco's grandma told him anybody could make the recipe.", "Franco made a whole meal for his family in one hour.", "Franco's family all loved the meal."]
}


In [ ]:
# Story 1

start = defaultdict(set)
start['Gina'] = set([stories[1][0]])
runStory(stories[1][1:], start)


In [ ]:
# Story 2

start = defaultdict(set)
start['Phil'] = set([stories[2][0]])
runStory(stories[2][1:], start)


In [ ]:
# Story 3

start = defaultdict(set)
start['Amy'] = set([stories[3][0]])
runStory(stories[3][1:], start)


In [ ]:
# Story 4

start = defaultdict(set)
start['Xander'] = set([stories[4][0]])
runStory(stories[4][1:], start)


In [ ]:
# Story 5

start = defaultdict(set)
start['Franco'] = set([stories[5][0]])
runStory(stories[5][1:], start)
